In [28]:
# Imports
import os
import asyncio
from typing import cast
from dotenv import load_dotenv
from typing import Dict
from agents import Agent, Runner, OpenAIChatCompletionsModel, AsyncOpenAI
from openai.types.responses import ResponseTextDeltaEvent


# Arize Phoenix Tracer
from phoenix.otel import register
from opentelemetry import trace as otel_trace


# Markdown output display
from IPython.display import Markdown, display

In [29]:
# Cast to String
def env_to_str(env: str) -> str:
    return cast(str, os.getenv(env))

In [30]:
# Load Environment Variables
load_dotenv(override=True)

# Ollama Environment Variables
ollama_api_key = env_to_str('OLLAMA_API_KEY')
ollama_base_url = env_to_str('OLLAMA_BASE_URL')
model_llama = env_to_str('MODEL_LLAMA3')

# Phoenix Collector Variables
phoenix_collector_endpoint = env_to_str("PHOENIX_COLLECTOR_ENDPOINT") or "http://localhost:6006/v1/traces"

# Maileroo Environment Variables
maileroo_api_key = env_to_str("MAILEROO_API_KEY")
maileroo_base_url = env_to_str("MAILEROO_BASE_URL")
maileroo_template_url = env_to_str("MAILEROO_TEMPLATE_URL")

In [31]:
# Agent Workflow Instructions

instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [32]:
# Configure Phoenix Tracer
try:
    tracer_provider = register(
    project_name="ollama_phoenix",
    auto_instrument=True,
    endpoint=phoenix_collector_endpoint,
    set_global_tracer_provider=True
)
    display(Markdown("✅ Phoenix tracer registered successfully"))
except Exception as e:
    display(Markdown(f"❌ Phoenix registration failed: {e}"))

Overriding of current TracerProvider is not allowed
DependencyConflict: requested: "openai-agents >= 0.1.0" but found: "openai-agents 0.0.17"
DependencyConflict: requested: "google-genai" but found: "None"


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: ollama_phoenix
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://localhost:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



✅ Phoenix tracer registered successfully

In [33]:
# Ollama Model
ollama_model = OpenAIChatCompletionsModel(
    model=model_llama,
    openai_client=AsyncOpenAI(
        api_key=ollama_api_key, 
        base_url=ollama_base_url
    )
)

In [34]:
# Agents

sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model=ollama_model
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model=ollama_model
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model=ollama_model
)

sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model=ollama_model
)

In [40]:
# Phoenix Tracer
async def phoenix_tracer(trace_name, message, *agents):
    tracer = otel_trace.get_tracer(__name__)
    with tracer.start_as_current_span(trace_name) as current_span:
        if agents:
            
            # Trace Attributes
            current_span.set_attribute("user.request", message)
            current_span.set_attribute("model.name", model_llama)
            
            for index, agent in enumerate(agents):
                
                # Trace Attributes
                current_span.set_attribute("agent.name", agent.name)
        
                results = await asyncio.gather(
                    *[Runner.run(agent, message) for agent in agents]
                )

                outputs = []

                for result in results:
                    outputs.append(result.final_output)

                combined_outputs = "\n\n--- Option ---\n\n".join(outputs)
                best = await Runner.run(agents[-1], combined_outputs)

                if outputs:
                    total_length = sum(len(output) for output in outputs)
                    
                    # Add response attributes
                    current_span.set_attribute("response.length", total_length)
                    # current_span.set_attribute("response.preview", outputs[0][:100] if outputs[0] else "")
                    current_span.set_attribute("response.preview", outputs[0] if outputs[0] else "")
        else:
            print("No additional arguments provided.")

        # outputs = [result.final_output for result in results]
        
        
        for output in outputs:
            display(Markdown(f"# Sales Agent:\n\n{output}\n\n"))
            display(Markdown(f"# Sales Picker:\n\n{best}"))
        return outputs, best

In [41]:
# Result
result, best = await phoenix_tracer(
    "Parallel Cold Emails", 
    "Write a cold sales email.", 
    sales_agent1, 
    sales_agent2, 
    sales_agent3,
    sales_picker
)

# Sales Agent:

Subject: Enhance Your Organization's Compliance Readiness with ComplAI

Dear [Decision Maker],

As a seasoned executive in the [industry/field], you understand the importance of maintaining strong compliance controls to protect your organization and its stakeholders. With the ever-increasing demands from regulators, auditors, and the market at large, ensuring SOC 2 compliance can be an uphill battle.

ComplAI's advanced SaaS platform utilizes cutting-edge AI technology to empower organizations like yours with automated control monitoring, customizable reporting, and intuitive audit preparation tools. Our goal is to ease your team's burden while elevating your company's risk posture.

Key benefits of our solution include:

- **Efficient Control Monitoring**: Continuously verifies SOC 2 compliance requirements, ensuring timely updates and accurate documentation.
- **Enhanced Security Posture**: Advanced analytics detects vulnerabilities and weaknesses before they become major issues.
- **Simplified Audit Preparation**: Comprehensive tools make the audit process more manageable, reducing stress and costs related to preparation.

Would you like to schedule a briefing to discuss how ComplAI can optimize your compliance strategy?

Best regards,
[Your Name]
ComplAILiaison | Phone: [Your Contact Number]



# Sales Picker:

RunResult:
- Last agent: Agent(name="sales_picker", ...)
- Final output (str):
    Subject: Enhance Your Organization's Compliance Readiness with ComplAI
- 1 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)

# Sales Agent:

Here's a cold sales email with some humor:

Subject: Your Audit is No laughing Matter (unless you're prepared)

Hi [Name],

I'm guessing you're not thrilled about the thought of your upcoming SOC 2 audit. Newsflash: audits are like awkward family reunions, but without the free food.

ComplAI to the rescue! Our AI-powered SaaS tool ensures your organization is SOC 2 compliant and prepared for those pesky auditors. No more scrambling around trying to find that elusive piece of documentation or praying you remember all the required security protocols.

Our platform helps you:

Streamline compliance efforts
Identify potential issues before they become major headaches
Increase audit success rates (read: reduce your risk of a sleepless night)

Want to learn more about how ComplAI can make auditing less painful? Let's chat! I'd be happy to schedule a demo and show you why our tool is the 'audit whisperer' for many companies like yours.

Hit reply or schedule a time that works for you: [Calendly Link]

Looking forward to helping you ace your next audit!

Cheers,
[Your Name]
ComplAI Sales & Support

P.S. We've got a 14-day free trial waiting for you, if you'd rather take it for a spin first. Just sayin'.

How's that?



# Sales Picker:

RunResult:
- Last agent: Agent(name="sales_picker", ...)
- Final output (str):
    Subject: Enhance Your Organization's Compliance Readiness with ComplAI
- 1 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)

# Sales Agent:

Here's a direct and concise cold email:

Subject: Simplify Your SOC 2 Compliance Journey

Hi [Recipient],

As a leader in [Industry], I assume you're aware of the growing importance of cybersecurity and compliance for your organization.

At ComplAI, we've developed an AI-powered SaaS tool that streamlines the time-consuming process of achieving SOC 2 compliance. Our platform automates audit preparation, saving you months of manual work and costly delays.

Would you like to schedule a quick call to discuss how our solution can help alleviate the burden of compliance on your team?

Best,
[Your Name]
ComplAI

This email is short, relevant, and attention grabbing. It:

1. Acknowledges the recipient's industry and their likely awareness of cybersecurity's importance.
2. Highlights the value proposition: saving time, manual labor, and costly delays.
3. Suggests a personal connection with the recipient (a quick call).
4. Ends with a clear next step to further discuss the solution.

Let me know if you want any variations or adjustments!



# Sales Picker:

RunResult:
- Last agent: Agent(name="sales_picker", ...)
- Final output (str):
    Subject: Enhance Your Organization's Compliance Readiness with ComplAI
- 1 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)

# Sales Agent:

Here's a sample cold sales email:

Subject: Boost Your Website's Visibility

Hi John,

I came across your website and noticed that it's ranking #4 for "best coffee shops in [City]". However, our recent study shows that websites with a SEO audit and optimization can increase their visibility by 300% and drive more quality leads.

Would you be open to a complimentary SEO review of your website? We'd love to help you maximize your online presence and reach a wider audience.

Best,
Emily
Founder, OptimizerX

This email aims to grab the recipient's attention with relevant information (his website's ranking) and offers a valuable solution to improve it. The tone is professional yet friendly, and the offer is complimentary, which makes it difficult for the recipient to say no.



# Sales Picker:

RunResult:
- Last agent: Agent(name="sales_picker", ...)
- Final output (str):
    Subject: Enhance Your Organization's Compliance Readiness with ComplAI
- 1 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)